In [1]:
# %pip install -r requirements.txt

Import required libraries

In [2]:
import pandas as pd

Load data

In [3]:
ald = pd.read_csv('inputs/ald_to_naturesense_with_companies_2024-11-26.csv')

/var/folders/dv/7my5mkxn2bv_gtglwkkjfbww0000gr/T/ipykernel_24465/3189856878.py:1: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  ald = pd.read_csv('inputs/ald_to_naturesense_with_companies_2024-11-26.csv')


In [4]:
ald.columns

Index(['asset_id', 'entity_name', 'entity_isin', 'latitude', 'longitude',
       'is_priority_location', 'country', 'country_code', 'realm', 'biome',
       'ecosystem_functional_group', 'river_basin', 'nature_risk_index',
       'sensitive_locations', 'biodiversity_importance_aggregated',
       'high_ecosystem_integrity_aggregated',
       'decline_in_ecosystem_integrity_aggregated',
       'physical_water_risk_aggregated',
       'ecosystem_services_provision_importance_aggregated',
       'proximity_to_protected_areas', 'proximity_to_kbas',
       'species_rarity_weighted_richness', 'species_threat_abatement',
       'species_threat_abatement_marine', 'proximity_to_mangroves',
       'ecosystem_integrity_index', 'biodiversity_intactness_index',
       'ocean_health_index', 'trend_in_ecosystem_integrity_index',
       'deforestation_hotspots', 'water_availability', 'water_pollution',
       'drought', 'riverine_flood', 'coastal_flood',
       'cumulative_impact_on_oceans',
       'c

Clean data

In [5]:
ald.rename(columns=lambda x: x.replace('_aggregated', ''), inplace=True)

In [6]:
ald.rename(columns={
    'ecosystem_integrity_index': 'ecosystem_intactness_index',
    'trend_in_ecosystem_integrity_index': 'trend_in_ecosystem_intactness_index'
}, inplace=True)

In [7]:
ald['priority_asset'] = (ald['sensitive_locations'] > 0.6).astype(int)

Aggregate ALD to Equities (Company averages)

In [8]:
columns_to_average = [
    'sensitive_locations', 'biodiversity_importance', 'high_ecosystem_integrity', 'decline_in_ecosystem_integrity',
    'physical_water_risk', 'ecosystem_services_provision_importance', 'proximity_to_protected_areas', 'proximity_to_kbas',
    'species_rarity_weighted_richness', 'species_threat_abatement', 'species_threat_abatement_marine', 'proximity_to_mangroves',
    'ecosystem_intactness_index', 'biodiversity_intactness_index', 'ocean_health_index', 'trend_in_ecosystem_intactness_index',
    'deforestation_hotspots', 'water_availability', 'water_pollution', 'drought', 'riverine_flood', 'coastal_flood', 
    'cumulative_impact_on_oceans', 'critical_areas_for_biodiversity_and_ncp', 'areas_of_importance_for_biodiversity_and_climate'
]

equities = ald.groupby('entity_isin').agg(
    assets_count=('entity_isin', 'count'),
    priority_assets_count=('priority_asset', 'sum'),
    **{f"{col}": (col, lambda x: round(x.mean(skipna=True), 4)) for col in columns_to_average}
).reset_index()


In [9]:
equities

,entity_isin,assets_count,priority_assets_count,sensitive_locations,biodiversity_importance,high_ecosystem_integrity,decline_in_ecosystem_integrity,physical_water_risk,ecosystem_services_provision_importance,proximity_to_protected_areas,...,trend_in_ecosystem_intactness_index,deforestation_hotspots,water_availability,water_pollution,drought,riverine_flood,coastal_flood,cumulative_impact_on_oceans,critical_areas_for_biodiversity_and_ncp,areas_of_importance_for_biodiversity_and_climate
0,AEA000201011,9,2,0.5458,0.0332,0.3040,0.3163,0.7688,0.5740,0.0243,...,0.3163,0.0,0.8750,0.5625,0.5695,0.7278,0.4069,0.6022,0.7303,0.3111
1,AEA000701010,1,0,0.5343,0.0001,0.3609,0.0547,0.7996,0.5343,0.0000,...,0.0547,0.0,1.0000,0.5000,NaN,0.7996,0.4257,0.5410,0.7857,0.2828
2,AEA000801018,2,1,0.6060,0.0155,0.2843,0.0738,0.7996,0.6060,0.0000,...,0.0738,0.0,1.0000,0.5000,NaN,0.7996,0.4257,0.4562,1.0000,0.2121
3,AEA001001014,1,1,0.8497,0.0178,0.4562,0.1835,0.8497,1.0000,0.0000,...,0.1835,0.0,1.0000,0.5000,NaN,0.7996,0.4257,NaN,1.0000,NaN
4,AEA001901015,1,1,0.7983,0.2056,0.7983,0.0962,0.7996,0.3596,0.3869,...,0.0962,0.0,1.0000,0.5000,NaN,0.7996,0.4257,0.5766,0.5071,0.2121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9714,ZAE000282356,1,0,0.4167,0.0126,0.3230,0.4167,0.6024,0.2525,0.0000,...,0.4167,0.0,1.0000,0.2500,0.6024,0.2204,0.0000,NaN,0.0000,0.5050
9715,ZAE000296554,15,3,0.5758,0.4652,0.6538,0.4752,0.5368,0.4438,0.6546,...,0.4752,0.0,1.0000,0.4333,0.5358,0.1529,0.0000,NaN,0.2981,0.6316
9716,ZAE000298253,8,5,0.5831,0.4876,0.5715,0.3779,0.5925,0.4439,0.5108,...,0.3779,0.0,0.9375,0.2500,0.5925,0.2745,0.0000,NaN,0.2074,0.6805
9717,ZAE000314084,1,0,0.4628,0.0002,0.3266,0.4628,0.6024,0.2525,0.0000,...,0.4628,0.0,1.0000,0.2500,0.6024,0.2204,0.0000,NaN,0.0000,0.5050


Generate Equities country split

In [10]:
# Group by entity_isin and country_code, then count assets
equities_country_split = ald.groupby(['entity_isin', 'country_code']).size().reset_index(name='assets_count')

# Calculate the total count for each entity_isin
total_assets_per_equity = equities_country_split.groupby('entity_isin')['assets_count'].transform('sum')

# Compute the percentage
equities_country_split['percentage_of_assets_in_country'] = equities_country_split['assets_count'] / total_assets_per_equity

In [11]:
equities_country_split

,entity_isin,country_code,assets_count,percentage_of_assets_in_country
0,AEA000201011,ARE,7,0.875000
1,AEA000201011,SGP,1,0.125000
2,AEA000701010,ARE,1,1.000000
3,AEA000801018,ARE,1,1.000000
4,AEA001001014,ARE,1,1.000000
...,...,...,...,...
39828,ZAE000296554,AUS,2,0.133333
39829,ZAE000296554,ZAF,13,0.866667
39830,ZAE000298253,ZAF,8,1.000000
39831,ZAE000314084,ZAF,1,1.000000


Export

In [12]:
equities.to_csv("outputs/equities_naturesense.csv", index=False)

In [13]:
equities_country_split.to_csv("outputs/equities_country_split.csv", index=False)